In [3]:
from web3 import Web3
from hexbytes import HexBytes
import pandas as pd
import numpy as np
import rlp
import os
import requests
from web3 import Web3
from dotenv import load_dotenv

from datetime import datetime, timedelta, timezone

import matplotlib.pylab as plt
import numpy as np
import json
import logging
import typing
import requests
import sys
import time

import pandas as pd
from dotenv import load_dotenv
from web3 import Web3

load_dotenv()

True

In [4]:

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [5]:
def empty_alerts() -> pd.DataFrame:
        df_forta = pd.DataFrame(columns=['createdAt', 'name', 'protocol', 'scanNodeCount', 'findingType', 'source', 'labels', 'severity', 'metadata', 'alertId', 'description', 'addresses', 'contracts', 'hash'])
        return df_forta

def alerts_by_agent3(agent_id: str, chain_id:str, start_date: datetime, end_date: datetime) -> pd.DataFrame:
    url = "https://api.forta.network/graphql"
    chunk_size = 8000

    df_forta = empty_alerts()
    json_data = ""
    first_run = True
    count = 0
    while (json_data == "" or json_data['data']['alerts']['pageInfo']['hasNextPage']):
        query = """query exampleQuery {
                alerts(
                    input: {
                        CHUNKSIZE
                        AFTER_CLAUSE
                        BLOCK_RANGE_CLAUSE
                        BOT_CLAUSE
                        CHAIN_ID_CLAUSE
                    }
                ) {
                    pageInfo {
                    hasNextPage
                    endCursor {
                        alertId
                        blockNumber
                    }
                    }
                    alerts {
                    createdAt
                    name
                    protocol
                    scanNodeCount
                    findingType
                    source {
                        transactionHash
                        block {
                        number
                        chainId
                        }
                        bot {
                        id
                        }
                    }
                    labels {
                        entity
                        entityType
                        label
                        metadata
                        remove
                        confidence
                    }
                    severity
                    metadata
                    alertId
                    description
                    addresses
                    contracts {
                        address
                        name
                        projectId
                    }
                    hash
                    }
                }
                }"""

        after_clause = ""
        if(first_run is False):
            blockNumber = json_data['data']['alerts']['pageInfo']['endCursor']['blockNumber']
            alertId = json_data['data']['alerts']['pageInfo']['endCursor']['alertId']
            after_clause = """after: {{blockNumber:{0}, alertId:"{1}"}}""".format(blockNumber, alertId)

        # this is a bit hacky
        query = query.replace("CHUNKSIZE", f"first: {chunk_size},") 
        query = query.replace("AFTER_CLAUSE", after_clause)
        query = query.replace("BLOCK_RANGE_CLAUSE", """blockDateRange: {{ startDate: "{0}", endDate: "{1}" }}""".format(datetime.strftime(start_date, "%Y-%m-%d"), datetime.strftime(end_date, "%Y-%m-%d")))
        query = query.replace("BOT_CLAUSE", f"""bots: ["{agent_id}"]""")
        query = query.replace("CHAIN_ID_CLAUSE", f"chainId: {chain_id}")

        #print(query)

        retries = 1
        wait = 1
        success = False
        while not success:
            try:
                count += 1
                forta_api_key = os.environ.get('FORTA_API_KEY')
                headers = {"Authorization": "Bearer " + forta_api_key}
                r = requests.post(url, json={'query': query}, headers=headers)
                if r.status_code == 200:
                    success = True
                    if chunk_size < 5000:
                        chunk_size *= 2
                        logging.warning(f"Increasing chunk size to {chunk_size}")
                else:
                    raise Exception(f"status code: {r.status_code} {r.text}")
            except Exception as e:
                logging.warning(f"Unable to retrieve alerts {r.status_code} , {e}")
                logging.warning(f"Sleeping {wait}sec. Count {count}.")
                old_chunk_size = chunk_size
                chunk_size = int(chunk_size/2)
                if(chunk_size<1):
                    chunk_size = 1
                query = query.replace(f"first: {old_chunk_size},", f"first: {chunk_size},") 
                logging.warning(f"Reducing chunk size to {chunk_size}")
                time.sleep(wait)
                retries += 1
                if retries > 30:
                    raise Exception("Unable to retrieve alerts even after repeated retries. Pls check logs")

        json_data = json.loads(r.text)
        df_data = json_data['data']['alerts']['alerts']
        df_forta = pd.concat([pd.DataFrame(df_data), df_forta])

        first_run = False
        count += 1

    return df_forta



In [6]:
start_date = datetime(2023, 11, 1, 0, 0, 0, 0, timezone.utc)
end_date = datetime(2023, 12, 12, 0, 0, 0, 0, timezone.utc)
filename = "AD3_52_Dec2023_Analysis"

In [7]:
source_id = "0x3172685467b021a6e6b9b0080edbf26e98d37eecd1ac90e89a8fa73b26e04e51" # attack detector beta alt
df_forta_alerts = empty_alerts()
df_forta_alerts['chain_id'] = pd.Series()
for chain_id in [ 1, 56, 137, 43114, 42161, 10, 250]:
    print(f"chain_id: {chain_id}")
    df_forta_alerts_temp = alerts_by_agent3(source_id, str(chain_id), start_date, end_date)
    df_forta_alerts_temp['chain_id'] = chain_id
    print(f"chain_id: {chain_id}, alerts: {len(df_forta_alerts_temp)}")
    df_forta_alerts = pd.concat([df_forta_alerts, df_forta_alerts_temp])
df_forta_alerts.to_csv(f"{filename}AttackDetector_beta2.csv")


chain_id: 1
2023-12-12 15:06:33,867 - root - WARNING - Unable to retrieve alerts 500 , status code: 500 {"message": "Internal server error"}
2023-12-12 15:06:33,868 - root - WARNING - Sleeping 1sec. Count 1.
2023-12-12 15:06:33,869 - root - WARNING - Reducing chunk size to 4000
2023-12-12 15:06:38,445 - root - WARNING - Unable to retrieve alerts 500 , status code: 500 {"message": "Internal server error"}
2023-12-12 15:06:38,449 - root - WARNING - Sleeping 1sec. Count 2.
2023-12-12 15:06:38,450 - root - WARNING - Reducing chunk size to 2000
2023-12-12 15:06:41,217 - root - WARNING - Increasing chunk size to 4000
2023-12-12 15:06:44,762 - root - WARNING - Unable to retrieve alerts 500 , status code: 500 {"message": "Internal server error"}
2023-12-12 15:06:44,763 - root - WARNING - Sleeping 1sec. Count 5.
2023-12-12 15:06:44,763 - root - WARNING - Reducing chunk size to 2000
2023-12-12 15:06:47,226 - root - WARNING - Increasing chunk size to 4000
2023-12-12 15:06:53,316 - root - WARNING 

In [8]:
source_id = "0x80ed808b586aeebe9cdd4088ea4dea0a8e322909c0e4493c993e060e89c09ed1" # attack detector
df_forta_alerts = empty_alerts()
df_forta_alerts['chain_id'] = pd.Series()
for chain_id in [ 1, 56, 137, 43114, 42161, 10, 250]:
    print(f"chain_id: {chain_id}")
    df_forta_alerts_temp = alerts_by_agent3(source_id, str(chain_id), start_date, end_date)
    df_forta_alerts_temp['chain_id'] = chain_id
    print(f"chain_id: {chain_id}, alerts: {len(df_forta_alerts_temp)}")
    df_forta_alerts = pd.concat([df_forta_alerts, df_forta_alerts_temp])
df_forta_alerts.to_csv(f"{filename}AttackDetector.csv")


chain_id: 1
2023-12-12 15:12:44,578 - root - WARNING - Unable to retrieve alerts 500 , status code: 500 {"message": "Internal server error"}
2023-12-12 15:12:44,582 - root - WARNING - Sleeping 1sec. Count 1.
2023-12-12 15:12:44,583 - root - WARNING - Reducing chunk size to 4000
2023-12-12 15:12:50,723 - root - WARNING - Unable to retrieve alerts 500 , status code: 500 {"message": "Internal server error"}
2023-12-12 15:12:50,725 - root - WARNING - Sleeping 1sec. Count 2.
2023-12-12 15:12:50,726 - root - WARNING - Reducing chunk size to 2000
2023-12-12 15:12:55,631 - root - WARNING - Unable to retrieve alerts 500 , status code: 500 {"message": "Internal server error"}
2023-12-12 15:12:55,634 - root - WARNING - Sleeping 1sec. Count 3.
2023-12-12 15:12:55,635 - root - WARNING - Reducing chunk size to 1000
2023-12-12 15:13:01,871 - root - WARNING - Unable to retrieve alerts 500 , status code: 500 {"message": "Internal server error"}
2023-12-12 15:13:01,875 - root - WARNING - Sleeping 1sec. 